In [5]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator
from sklearn.model_selection import (
    cross_validate,
    StratifiedKFold,
    KFold,
    StratifiedGroupKFold,
)
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.preprocessing import (
    SplineTransformer,
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
    PolynomialFeatures
)
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA


from bayes_opt import BayesianOptimization

from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor, kernels
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor


from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
import numpy as np

In [6]:
train = pd.read_csv("../data/train.csv", index_col="SEQN")
test_data = pd.read_csv("../data/test.csv", index_col="SEQN")

train_x, train_y = train.drop("y", axis=1), train["y"]

In [7]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)


train

,self_eval,teacher_eval,extracurricular,district,SRP_1,SRP_2,SRP_3,SRP_4,SRP_5,SRP_6,...,SRP_42,SRP_43,SRP_44,SRP_45,SRP_46,SRP_47,SRP_48,SRP_49,SRP_50,y
SEQN,,,,,,,,,,,,,,,,,,,,,
969167,4,5,9,3,-0.181,-0.379,-0.164,0.080,0.378,1.581,...,-1.156,-0.730,-0.508,-0.497,0.224,0.412,-0.517,0.099,0.114,-1.315
188942,4,3,5,4,-0.126,1.603,1.021,0.489,-1.404,-0.955,...,-0.318,1.240,-1.993,2.021,-1.078,-0.277,0.802,0.253,-0.720,1.997
134058,1,2,8,5,0.724,-0.702,2.249,0.910,0.330,0.411,...,0.449,1.980,-0.401,-0.544,-0.944,1.592,0.875,-0.734,-2.336,3.709
124022,3,3,10,6,0.706,-0.302,1.023,-0.895,0.625,1.283,...,2.025,-2.289,-0.407,0.025,-0.515,0.408,1.380,-1.075,-2.451,1.155
685285,5,5,1,5,-0.350,-1.001,0.931,0.192,0.491,0.292,...,-0.118,-0.288,0.457,-0.566,0.822,-0.317,0.661,2.096,0.004,-1.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970998,2,1,2,1,1.035,1.359,1.558,-1.530,-0.220,0.075,...,2.717,-1.184,-0.300,0.444,0.024,-0.745,-0.890,-1.192,-1.083,-0.139
971286,5,3,1,2,-0.236,-0.723,-1.624,-1.306,0.783,-0.105,...,1.417,-0.061,-0.123,-1.063,-0.128,2.908,-0.057,-0.303,-0.150,0.394
852862,4,3,2,5,0.233,-1.349,-0.876,-0.544,0.400,0.381,...,0.183,-0.480,-2.044,0.084,0.504,1.913,-0.471,0.454,0.691,0.597


In [8]:
scorer = make_scorer(r2_score)

pca_pipeline = Pipeline(
    steps=[
        (
            "scale",
            StandardScaler(),
        ),
        # (
        #     "scrp_pca",
        #     PCA(n_components=10),
        # ),
    ]
)

numeric_preprocessor = ColumnTransformer(
    [
        (
            "scale",
            StandardScaler(),
            ["self_eval", "teacher_eval", "extracurricular"]
            + [f"SRP_{j}" for j in range(1, 51)],
        ),
        ("district", "passthrough", ["district"]),
        # ("pca", pca_pipeline, [f"SRP_{j}" for j in range(1, 51)]),
    ],
    remainder="drop",
)

scaling = Pipeline(
    [
        ("num_preprocess", numeric_preprocessor),
        # ("spline", SplineTransformer())
    ]
)


# categorical_encoder = ColumnTransformer([
#     ("cat", OrdinalEncoder(), ["district"])
# ])


# preprocessor = FeatureUnion([
#     ("numeric", scaling),
#     # ("categorical", categorical_encoder)
# ])

preprocessor = numeric_preprocessor
# new_train_data = preprocessor.fit_transform(train)

In [41]:
# preprocessor 2
class RowSum(BaseEstimator):

    def __init__(self) -> None:
        super().__init__()

    def fit(self, *args, **kwargs):
        pass

    def transform(self, X, y=None, **kwargs):
        if isinstance(X, pd.DataFrame):
            return X.values.sum(axis=1).reshape(-1, 1)
        else:
            return X.sum(axis=1).reshape(-1, 1)

    def fit_transform(self, X, y=None, **kwargs):
        return self.transform(X, y, **kwargs)

    def get_feature_names_out(self, X, y=None, **kwargs):
        return ["rowSums"]


# sumRows = Pipeline(
#     [
#         ("sums", RowSum()),
#         # ("scale", StandardScaler()),
#     ]
# )

colTrans = ColumnTransformer(
    [
        # ("one_hot", OneHotEncoder(sparse_output=False, drop="first"), ["district"]),
        ("collapse", RowSum(), [f"SRP_{i}" for i in range(1, 51)]),
        # ("select", "passthrough", ["self_eval", "teacher_eval", "extracurricular"])
    ],
    remainder="passthrough",
)

# numeric_prep = Pipeline(
#     [
#         ("setup", colTrans),
#         ("polys", PolynomialFeatures(interaction_only=True)),
#         # ("rescale", StandardScaler()),
#         # ("splines", SplineTransformer(knots="quantile", n_knots=10, extrapolation="linear")),
#     ]
# )


# preprocessor = FeatureUnion([
#     ("select_district", ColumnTransformer([("district", "passthrough", ["district"])])),
#     ("numeric", numeric_prep)
# ])
preprocessor = colTrans
preprocessor.fit_transform(train_x)

array([[-3.336,  4.   ,  5.   ,  9.   ,  3.   ],
       [ 8.854,  4.   ,  3.   ,  5.   ,  4.   ],
       [13.495,  1.   ,  2.   ,  8.   ,  5.   ],
       ...,
       [ 7.471,  4.   ,  3.   ,  2.   ,  5.   ],
       [13.44 ,  5.   ,  5.   ,  5.   ,  7.   ],
       [-3.596,  5.   ,  5.   ,  2.   ,  5.   ]])

In [43]:
def optim(
    learning_rate=0.1,
    max_iter=10,
    l2_regularization=1,
    max_bins=50,
    max_depth=4,
    max_leaf_nodes=5,
    return_model=False,
):

    scorer = make_scorer(r2_score)
    max_depth = int(max_depth)
    max_bins = int(max_bins)
    max_leaf_nodes = int(max_leaf_nodes)
    max_iter = int(max_iter)

    model = HistGradientBoostingRegressor(
        learning_rate=learning_rate,
        max_iter=max_iter,
        l2_regularization=l2_regularization,
        max_bins=int(max_bins),
        categorical_features=[4],
        max_leaf_nodes=max_leaf_nodes,
        max_depth=max_depth,
    )

    model_pipeline = Pipeline([("preprocess", preprocessor), ("model", model)])

    # TODO: add cross manual cross validation here within district
    if return_model:
        return (
            cross_validate(
                model_pipeline,
                X=train_x,
                y=train_y,
                scoring=scorer,
                cv=KFold(shuffle=True),
            ),
            model_pipeline,
        )

    else:
        return cross_validate(
            model_pipeline,
            X=train_x,
            y=train_y,
            scoring=scorer,
            cv=KFold(shuffle=True, n_splits=5),
        )["test_score"].mean()


In [44]:
optimizer = BayesianOptimization(
    optim,
    pbounds={
        "learning_rate": [0.01, 0.30],
        "max_iter": [1000, 4000],
        "l2_regularization": [1, 4000],
        "max_bins": [5, 100],
        "max_leaf_nodes": [3, 25],
        "max_depth": [2, 50],
    },
)

optimizer.maximize(init_points=7, n_iter=35)

|   iter    |  target   | l2_reg... | learni... | max_bins  | max_depth | max_iter  | max_le... |
-------------------------------------------------------------------------------------------------
| 1         | 0.8746    | 844.8     | 0.3365    | 28.26     | 42.16     | 1.408e+03 | 13.09     |
| 2         | 0.8817    | 3.115e+03 | 0.1419    | 31.55     | 39.06     | 2.604e+03 | 5.042     |
| 3         | 0.8731    | 358.1     | 0.4377    | 22.88     | 40.47     | 1.187e+03 | 7.494     |
| 4         | 0.8391    | 3.972e+03 | 0.1192    | 68.83     | 47.0      | 173.3     | 17.45     |
| 5         | 0.882     | 3.592e+03 | 0.2364    | 61.34     | 26.98     | 1.15e+03  | 16.85     |
| 6         | 0.8825    | 3.71e+03  | 0.2543    | 60.51     | 13.81     | 1.095e+03 | 12.35     |
| 7         | 0.8806    | 3.96e+03  | 0.3592    | 42.53     | 46.23     | 1.835e+03 | 9.626     |
| 8         | 0.8744    | 3.111e+03 | 0.4052    | 20.59     | 26.43     | 2.582e+03 | 11.45     |
| 9         | 0.8764

In [68]:
optimal_params = optimizer.max["params"]
for param in ["max_bins", "max_depth", "max_iter", "max_leaf_nodes"]:
    optimal_params.update({param: int(optimal_params[param])})

test_model = HistGradientBoostingRegressor(
    **optimal_params, categorical_features=[len(train_x.columns) - 1]
)

model_pipeline = Pipeline([("preprocess", preprocessor), ("model", test_model)])
cv_results = cross_validate(
    model_pipeline, X=train_x, y=train_y, scoring=scorer, cv=KFold(shuffle=True)
)

model_pipeline.fit(train_x, train_y)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  ['self_eval', 'teacher_eval',
                                                   'extracurricular', 'SRP_1',
                                                   'SRP_2', 'SRP_3', 'SRP_4',
                                                   'SRP_5', 'SRP_6', 'SRP_7',
                                                   'SRP_8', 'SRP_9', 'SRP_10',
                                                   'SRP_11', 'SRP_12', 'SRP_13',
                                                   'SRP_14', 'SRP_15', 'SRP_16',
                                                   'SRP_17', 'SRP_18', 'SRP_19',
                                                   'SRP_20', 'SRP_21', 'SRP_22',
                                                   'SRP_23', 'SRP_24', 'SRP_25',
                                                   'SRP_26', 'SRP_27', ...]),
                                                 ('district', 'passthrough',
                                                  ['district'])])),
                ('model',
                 HistGradientBoostingRegressor(categorical_features=[53],
                                               l2_regularization=339.36642890954886,
                                               learning_rate=0.13475235198092747,
                                               max_bins=59, max_depth=16,
                                               max_iter=1895,
                                               max_leaf_nodes=4))])

In [75]:
# looks good, so spit out the results
test_data["y"] = model_pipeline.predict(test_data)
test_data["y"].to_csv("../results/cv_results.csv")